# Unet Review
---
description

### Sections:
- a
- b

---

## Initialization

install required packages using <code>requirements.txt</code>

In [ ]:
!. ../venv/bin/activate

In [ ]:
!python3 -mpip install pip --upgrade
!python3 -mpip install -r ./requirements.txt

## Import packages

In [4]:
from __future__ import division
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

# 
from matplotlib import pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import cv2
from scipy.ndimage.morphology import binary_erosion, binary_dilation
from scipy.ndimage.morphology import binary_fill_holes, binary_opening

# 
from tqdm import tqdm_notebook as tqdm
import json
import copy
import glob

In [ ]:
from extra.utils import (
  load_config,
  _print,
)

## Load a configuration

In [ ]:
CONFIG_FILE_PATH = "./configs/default.yaml"

In [ ]:
config = load_config(CONFIG_FILE_PATH)
_print("Config:", "info_underline")
print(json.dumps(config, indent=2))
print(20*"~-", "\n")

In this section, the required configuration and static variable are loaded.

In [ ]:
with open(CONFIG_FILE_PATH, 'r') as fp:
    config = json.load(fp)

Path(config['DATA']['NPY_DIR']).mkdir(parents=True, exist_ok=True)
    
run_name = config['RUN']['name']
imsk_fn = config['RUN']['imsk_folder_name']
final_imsk_fn = config['RUN']['final_imsk_folder_name']

Path('./results').mkdir(parents=True, exist_ok=True)
Path('./results/run_{}'.format(run_name)).mkdir(parents=True, exist_ok=True)
Path('./results/run_{}/{}'.format(run_name, imsk_fn)).mkdir(parents=True, exist_ok=True)
Path('./results/run_{}/{}'.format(run_name, final_imsk_fn)).mkdir(parents=True, exist_ok=True)
Path('./results/run_{}/{}/filtered'.format(run_name, imsk_fn)).mkdir(parents=True, exist_ok=True)

IMSK_DIR   = './results/run_{}/{}/'        .format(run_name, imsk_fn)
F_IMSK_DIR = './results/run_{}/{}/filtered/'.format(run_name, imsk_fn)

CROP_SCALE_LIST = config['RUN']['crop_scale_list']
for sc in CROP_SCALE_LIST+[0,]:
    Path('{}sc_{:1.2f}'.format(  IMSK_DIR, sc)).mkdir(parents=True, exist_ok=True)
    Path('{}sc_{:1.2f}'.format(F_IMSK_DIR, sc)).mkdir(parents=True, exist_ok=True)

## Set global variables

In [ ]:
number_classes = int(config['dataset']['number_classes'])
input_channels = 3
best_val_loss  = np.inf
patience       = 0
device         = 'cuda' if torch.cuda.is_available() else 'cpu'
data_path      = config['dataset']['root_directory']

## Develop some required functions

In [1]:
def get_biggest_cc_msk(mask):
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=4)
    sizes = stats[:, -1]
    
    if len(sizes)<2:
        return mask
    
    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

    img2 = np.zeros(output.shape)
    img2[output == max_label] = 255
    return img2


def show_sbs(iml, imr):
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.imshow(iml, interpolation='none')
    plt.subplot(1,2,2)
    plt.imshow(imr, interpolation='none')
    return

## Load the dataset

In [ ]:
from datasets import ISIC2018Dataset

In [ ]:
subjects_tr, subjects_vl = train_test_split(subjects, test_size=config['validation_p'], shuffle=True)

tr_dataset = ISIC2018Dataset(subjects=subjects_tr, output_size=output_size)
tr_loader  = DataLoader(tr_dataset, batch_size=int(config['train']['batch_size']), shuffle=True)

vl_dataset = ISIC2018Dataset(subjects=subjects_vl, output_size=output_size)
vl_loader  = DataLoader(vl_dataset, batch_size=int(config['validation']['batch_size']), shuffle=True)

ts_dataset = ISIC2018Dataset(subjects=subjects_ts, output_size=output_size)
ts_loader  = DataLoader(ts_dataset, batch_size=int(config['test']['batch_size']), shuffle=True)

## Load the model

In [ ]:
device = torch.device(device)

In [ ]:
from models import XDV

Network = XDV(pretrained=True).to(device)

if int(config['model']['pretrained']):
    Network.load_state_dict(torch.load(config['save_path'], map_location='cpu')['model_weights'])
    best_val_loss = torch.load(config['save_path'], map_location='cpu')['val_loss']

## Determine parameters

In [ ]:
optimizer = optim.Adam(
                Network.parameters(), 
                lr=float(config['training']['lr'])
            )

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, 
                'min', 
                factor=0.5, 
                patience=config['training']['patience']
            )

criteria  = torch.nn.BCELoss()

## Train the network

In [ ]:
print('Started training phase.') 
for ep in range(int(config['training']['epochs'])):
    Network.train()
    epoch_loss = 0
    for itter, batch in enumerate(tr_loader):
        img = batch['x'].to(device, dtype=torch.float)
        msk = batch['y'].to(device)
        
        pred = Network(img)
        loss = criteria(pred, msk) 
        optimizer.zero_grad()
        loss.backward()
        
        epoch_loss += loss.item()
        optimizer.step()  
        
        if itter%int(float(config['training']['progress_p']) * len(tr_loader))==0:
            print(f'Epoch ({ep+1:03d}), itteration {itter+1:04d}: loss = {((epoch_loss/(itter+1)):0.5f)}')
            
    ## Validation phase
    with torch.no_grad():
        print('Validation')
        val_loss = 0
        Network.eval()
        for itter, batch in enumerate(vl_loader):
            img = batch['x'].to(device, dtype=torch.float)
            msk = batch['y'].to(device)

            pred = Network(img)
            loss = criteria(pred, msk) 

            val_loss += loss.item()
        print(f' Validation on epoch {ep+1:03d}: Dice loss = {(abs(val_loss/(itter+1)):0.5f)}')     
        mean_val_loss = (val_loss/(itter+1))
        
        # Check the performance and save the model
        if (mean_val_loss) < best_val_loss:
            print('New best loss, saving...')
            best_val_loss = copy.deepcopy(mean_val_loss)
            state = copy.deepcopy({'model_weights': Network.state_dict(), 'val_loss': best_val_loss})
            torch.save(state, config['model']['save_path'])
            
    scheduler.step(mean_val_loss)
    
print('Training phase finished.')    

## Test

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from scipy.ndimage.morphology import binary_fill_holes, binary_opening

In [ ]:
predictions = []
gt = []

with torch.no_grad():
    print('val_mode')
    val_loss = 0
    Net.eval()
    for itter, batch in tqdm(enumerate(ts_loader)):
        img  = batch['x'].to(device, dtype=torch.float)
        msk  = batch['y']
        pred = Network(img)

        gt.append(msk.numpy()[0, 0])
        msk_pred = pred.cpu().detach().numpy()[0, 0]
        msk_pred = np.where(msk_pred>=0.43, 1, 0)
        msk_pred = binary_opening(msk_pred, structure=np.ones((6,6))).astype(msk_pred.dtype)
        msk_pred = binary_fill_holes(msk_pred, structure=np.ones((6,6))).astype(msk_pred.dtype)
        predictions.append(msk_pred)        

predictions = np.array(predictions)
gt = np.array(gt)

y_scores = predictions.reshape(-1)
y_true   = gt.reshape(-1)

y_scores2 = np.where(y_scores>0.47, 1, 0)
y_true2   = np.where(y_true>0.5, 1, 0)

## Evaluation

In [ ]:
#F1 score
F1_score = f1_score(y_true2, y_scores2, labels=None, average='binary', sample_weight=None)
print ("\nF1 score (F-measure) or DSC: " +str(F1_score))
confusion = confusion_matrix(np.int32(y_true), y_scores2)
print (confusion)
accuracy = 0
if float(np.sum(confusion))!=0:
    accuracy = float(confusion[0,0]+confusion[1,1])/float(np.sum(confusion))
print ("Accuracy: " +str(accuracy))
specificity = 0
if float(confusion[0,0]+confusion[0,1])!=0:
    specificity = float(confusion[0,0])/float(confusion[0,0]+confusion[0,1])
print ("Specificity: " +str(specificity))
sensitivity = 0
if float(confusion[1,1]+confusion[1,0])!=0:
    sensitivity = float(confusion[1,1])/float(confusion[1,1]+confusion[1,0])
print ("Sensitivity: " +str(sensitivity))

## Demonistration

## Benchmark

## Save results